## Convert a pytorch model to openvino

In [3]:
from torchvision.models import resnet50, ResNet50_Weights
import openvino as ov
import torch


# create model object
pytorch_model = resnet50(weights=ResNet50_Weights.DEFAULT)
# switch model from training to inference mode
pytorch_model.eval()

# Converting a model to OV format in float16
ov_model = ov.convert_model(pytorch_model.to(torch.float16), input=(ov.Type.f16, (1, 3, 224, 224)))

ov_model


<Model: 'Model964'
inputs[
<ConstOutput: names[x] shape[1,3,224,224] type: f16>
]
outputs[
<ConstOutput: names[x.45] shape[1,1000] type: f16>
]>

In [4]:
# Save model to disk (optional)
ov.save_model(ov_model, 'artifacts\\resnet50_fp16.xml')

In [5]:

core = ov.Core()
device = "NPU"

compiled_model = core.compile_model(model=ov_model, device_name=device)
output_layer = compiled_model.outputs[0]

result = compiled_model((torch.randn(1, 3, 224, 224, dtype=torch.float16),))[output_layer]

result.shape, result.dtype

((1, 1000), dtype('float16'))